## Load VALL

In [1]:
import gzip
import json
import pickle
import scipy
from sklearn.cluster import KMeans
from scipy import signal
import numpy as np

In [2]:
%%time
with gzip.GzipFile('../data/vall.jul19.2011.json.gz', 'r') as fin:
    json_bytes = fin.read() 

json_str = json_bytes.decode('utf-8')
vall = json.loads(json_str)
print("vall size: ", len(vall))

vall size:  16800
CPU times: user 4.81 s, sys: 439 ms, total: 5.25 s
Wall time: 5.25 s


## Load kmeans clusters

In [3]:
%%time
with open('../data/phipsi_km20.pkl', 'rb') as f:
    KM = pickle.load(f)
    
NCLUST = KM.cluster_centers_.shape[0]

CPU times: user 1.63 ms, sys: 19.4 ms, total: 21 ms
Wall time: 25.4 ms


## Assign clusters to (&phi;,&psi;) pairs

In [4]:
%%time
for key,item in vall.items():
    n = len(item['seq'])
    phi = np.array(item['phi'], dtype=np.float16)
    psi = np.array(item['psi'], dtype=np.float16)
    avec = np.vstack([np.sin(phi).T, np.cos(phi).T, np.sin(psi).T, np.cos(psi).T ]).T
    item['abin'] = np.eye(NCLUST)[KM.predict(avec)]

CPU times: user 15.8 s, sys: 698 ms, total: 16.5 s
Wall time: 16.5 s


In [5]:
# one of the vall 'profiles'
vall['7odcA']['abin']

array([[0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.]])

## Searching

In [9]:
# randomly generated 'profile'
L = 200
query = np.random.rand(L, NCLUST)


### One fragment

In [10]:
%%time

hits = {}

# search vall for one fragment
chunk = query[0:9,:]
for key,item in vall.items():
    hits[key] = signal.correlate2d(item['abin'], chunk, mode='valid')

print(hits['7odcA'].shape)

(379, 1)
CPU times: user 4.75 s, sys: 76 ms, total: 4.83 s
Wall time: 4.82 s


### All fragments

In [10]:
%%time

# search for 9-mers
WINDOW = 9

hits = {}

iter=0
for key,item in vall.items():
    iter += 1
    
    # print every 1000
    if iter%1000==0:
        print(iter, key)
    
    # correlate all fragments of the query
    hits[key] = np.hstack([signal.correlate2d(item['abin'], query[shift:shift+WINDOW,:], mode='valid') 
               for shift in range(L - WINDOW + 1)])


1000 2boyA
2000 3civA
3000 1kz1A
4000 3hygA
5000 1nezG
6000 2fupA
7000 2cioA
8000 3m5wA
9000 2pbnA
10000 1xu2R
11000 1u46A
12000 3pqkA
13000 3h0nA
14000 1tuwA
15000 3ofgA
16000 1fyeA
CPU times: user 2min 34s, sys: 595 ms, total: 2min 35s
Wall time: 2min 36s


In [12]:
hits['7odcA'].shape

(379, 92)

## Tensorflow

Superfast ```tensorflow``` search which takes ~1s to scan through entire VALL

In [6]:
import tensorflow as tf

print("Built with GPU:", tf.test.is_built_with_cuda())
print("GPU available:", tf.test.is_gpu_available())
print("GPU device:", tf.test.gpu_device_name())


Built with GPU: True
GPU available: True
GPU device: /device:GPU:0


In [7]:
vall_vec = np.vstack([item['abin'] for _,item in vall.items()])
vall_in = np.reshape(vall_vec, (1,vall_vec.shape[0],20,1))
vall_in.shape

(1, 4126307, 20, 1)

In [11]:
WINDOW = 9
tmp = np.stack([query[shift:shift+WINDOW,:] for shift in range(L - WINDOW + 1)])
filter = np.reshape(np.transpose(tmp, axes=[1,2,0]), (9,20,1,192))

In [14]:
%%timeit

tf.reset_default_graph()

i = tf.placeholder(tf.float32, (1, 4126307, 20, 1))
f = tf.placeholder(tf.float32, (9,20,1,192))
o = tf.nn.conv2d(input=i, filter=f, strides=[1,1,1,1], padding='VALID')

with tf.Session() as sess:
    out = sess.run(o, feed_dict={i : vall_in, f : filter})

978 ms ± 19.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
